In [0]:
import os
import requests
import json
import datetime


class FeeddexAPI:
    def __init__(self, root_url, organization):
        # Initialize the API with the root URL and organization name
        self.root_url = root_url
        self.organization = organization
        self.DATABRICKS_SECRET_SCOPE_ATM = "<ENTER DATABRICKS SCOPE"
        # Retrieve username and password from Databricks secret scope
        self.username = dbutils.secrets.get(
            scope=self.DATABRICKS_SECRET_SCOPE_ATM, key="<ENTER USERNAME AS PROVIDED BY FEEDDEX>")
        self.password = dbutils.secrets.get(
            scope=self.DATABRICKS_SECRET_SCOPE_ATM, key="<ENTER PASSWORD AS PROVIDED BY FEEDDEX>")
        self.version = "v1"
        # Authenticate and get the token
        self.token = self._authenticate()

    def _authenticate(self):
        # Prepare authentication data
        auth_data = {
            "authCode": self.username,
            "organization": self.organization,
            "password": self.password
        }
        # Send POST request to authenticate and get the token
        response = requests.post(
            os.path.join(self.root_url, "authenticate"),
            json=auth_data
        )
        response.raise_for_status()  # Raise an exception for error HTTP status codes
        return response.json()["token"]

    def get_feedback_data(self, year=datetime.datetime.now().strftime("%Y"), month=datetime.datetime.now().strftime("%m")):
        # Set up headers with the authorization token
        headers = {"Authorization": f"Bearer {self.token}"}
        # Set up parameters for the request
        params = {"year": year, "month": month}
        # Send GET request to retrieve feedback data
        r_data = requests.get(
            os.path.join(self.root_url, self.version, "feedbackData"),
            headers=headers,
            params=params
        )
        r_data.raise_for_status()  # Raise an exception for error HTTP status codes
        return r_data


# Define the root URL and organization
root_url = "https://api.feeddex.nl"
organization = "<ENTER ORGANIZATION NAME AS PROVIDED BY FEEDDEX>"
# Create an instance of the FeeddexAPI class
feeddex_api = FeeddexAPI(root_url, organization)
# Get feedback data
r_data = feeddex_api.get_feedback_data()
# Get the content of the response
content = r_data.content
# Parse the JSON content
d = json.loads(r_data.content)


In [0]:
from pyspark.sql.types import StringType, DateType, IntegerType, TimestampType, StructType, StructField, FloatType
from datetime import datetime
from pyspark.sql import functions as F
from pyspark.sql.functions import col, round


class FeedbackDataProcessor:
    def __init__(self, spark):
        # Initialize the FeedbackDataProcessor with a Spark session
        self.spark = spark

        # Define the schema for the feedback data
        self.feedback_schema = StructType([
            StructField("id", IntegerType(), True),
            StructField("date", DateType(), True),
            StructField("customerCode", IntegerType(), True),
            StructField("unitId", IntegerType(), True),
            StructField("enteredOn", TimestampType(), True),
            StructField("sentOn", TimestampType(), True),
            StructField("remindedOn", TimestampType(), True),
            StructField("feedbackStarted", TimestampType(), True),
            StructField("modelQuestionsCompleted", TimestampType(), True),
            StructField("feedbackCompleted", TimestampType(), True),
            StructField("expirationDate", DateType(), True),
            StructField("value", IntegerType(), True),
            StructField("valueAsGrade", FloatType(), True),
        ])

        # Define the schema for the unit data
        self.unit_schema = StructType([
            StructField("id", IntegerType(), True),
            StructField("name", StringType(), True)
        ])

    def parse_date(self, date_str, date_format="%Y-%m-%d"):
        # Parse a date string into a date object
        return datetime.strptime(date_str, date_format).date() if date_str else None

    def parse_timestamp(self, timestamp_str, timestamp_format="%Y-%m-%dT%H:%M:%S.%fZ"):
        # Parse a timestamp string into a datetime object
        return datetime.strptime(timestamp_str, timestamp_format) if timestamp_str else None

    def process_feedback_data(self, feedback_data):
        # Process the feedback data and create a DataFrame
        data = [
            (
                int(f["transaction"]["id"]),
                self.parse_date(f["transaction"]["date"]),
                int(f["transaction"]["customerCode"]),
                int(f["transaction"]["unitId"]),
                self.parse_timestamp(f["transaction"]["enteredOn"]),
                self.parse_timestamp(f["transaction"]["sentOn"]),
                self.parse_timestamp(f["transaction"].get("remindedOn")),
                self.parse_timestamp(f["transaction"].get("feedbackStarted")),
                self.parse_timestamp(f["transaction"].get(
                    "modelQuestionsCompleted")),
                self.parse_timestamp(
                    f["transaction"].get("feedbackCompleted")),
                self.parse_date(f["transaction"].get("expirationDate")),
                f["f100Scores"]["overallScore"]["value"] if f["f100Scores"] else None,
                f["f100Scores"]["overallScore"]["valueAsGrade"] if f["f100Scores"] else None
            )
            for f in feedback_data
        ]
        return self.spark.createDataFrame(data, self.feedback_schema)

    def process_unit_data(self, unit_data):
        # Process the unit data and create a DataFrame
        return self.spark.createDataFrame(unit_data, self.unit_schema)

    def join_dataframes(self, feedback_df, unit_data_df):
        # Join the feedback and unit data DataFrames
        return feedback_df.join(unit_data_df, feedback_df["unitId"] == unit_data_df["id"], "left") \
            .select(
                feedback_df["id"].alias("transactionId"),
                "date",
                "customerCode",
                "unitId",
                unit_data_df["name"].alias("region"),
                "enteredOn",
                "sentOn",
                "remindedOn",
                "feedbackStarted",
                "modelQuestionsCompleted",
                "feedbackCompleted",
                "expirationDate",
                "value",
                "valueAsGrade"
        )

    def calculate_summary(self, df):
        # Calculate summary statistics from the processed DataFrame
        return df.withColumn("year", F.substring(col("modelQuestionsCompleted").cast(StringType()), 1, 4)) \
                 .withColumn("month", F.substring(col("modelQuestionsCompleted").cast(StringType()), 6, 2)) \
                 .groupBy("unitId", "region", "year", "month") \
                 .agg(
                     round((F.sum(F.coalesce(col("value"), F.lit(0))) /
                           F.count(col("value"))), 2).alias("avgValue"),
                     round((F.sum(F.coalesce(col("valueAsGrade"), F.lit(0))) /
                           F.count(col("valueAsGrade"))), 2).alias("avgValueAsGrade"),
                     F.countDistinct(col("customerCode")).alias(
                         "customersThisPeriod"),
                     F.sum(F.when(col("valueAsGrade") > 8, 1).otherwise(
                         0)).alias("promotors"),
                     F.sum(F.when((col("valueAsGrade") >= 0) & (
                         col("valueAsGrade") <= 6), 1).otherwise(0)).alias("detractors"),
                     F.sum(F.when((col("valueAsGrade") > 6) & (
                         col("valueAsGrade") <= 8), 1).otherwise(0)).alias("passives")
        ) \
            .withColumn("questionnairesCompleted", col("promotors") + col("passives") + col("detractors")) \
            .withColumn("netPromotorScore", 100 * round((col("promotors") / col("questionnairesCompleted")) - (col("detractors") / col("questionnairesCompleted")), 2)) \
            .orderBy("unitId", "year", "month")

    def process_data(self, feedback_data, unit_data):
        # Process the feedback and unit data, join them, and calculate summary statistics
        feedback_df = self.process_feedback_data(feedback_data)
        unit_data_df = self.process_unit_data(unit_data)
        processed_df = self.join_dataframes(feedback_df, unit_data_df)
        summary_df = self.calculate_summary(processed_df)
        return processed_df, summary_df


# Assuming you have a SparkSession named "spark"
processor = FeedbackDataProcessor(spark)
processed_df, summary_df = processor.process_data(
    d["feedbackData"], d["unitData"])


In [0]:
display(processed_df)
display(summary_df)

transactionId,date,customerCode,unitId,region,enteredOn,sentOn,remindedOn,feedbackStarted,modelQuestionsCompleted,feedbackCompleted,expirationDate,value,valueAsGrade
50239214,2024-07-17,12166044,8725,Almere,2024-07-24T01:00:10.597354Z,2024-07-24T07:33:45.085481Z,2024-08-01T07:29:40.45418Z,2024-08-08T07:34:50.852548Z,2024-08-08T07:38:17.639062Z,2024-08-08T07:38:29.477736Z,2024-10-22,73,8.79
50239216,2024-07-15,23007015,8726,Amsterdam ZO,2024-07-24T01:00:11.524074Z,2024-07-24T07:33:46.886244Z,2024-08-01T07:29:41.77657Z,2024-08-01T08:31:45.177857Z,2024-08-01T08:33:33.946347Z,2024-08-01T08:33:56.891765Z,2024-10-22,100,10.0
50239217,2024-07-15,25107657,8726,Amsterdam ZO,2024-07-24T01:00:11.595473Z,2024-07-24T07:33:48.221145Z,2024-08-01T07:29:42.742834Z,2024-08-05T12:29:14.581742Z,2024-08-05T12:30:35.328569Z,2024-08-05T12:30:42.949277Z,2024-10-22,53,7.9
50239223,2024-07-19,11129380,8726,Amsterdam ZO,2024-07-24T01:00:12.11311Z,2024-07-24T07:33:47.387411Z,2024-08-01T07:29:46.398075Z,2024-08-06T20:34:17.047282Z,2024-08-06T20:36:02.459031Z,2024-08-06T20:36:15.366102Z,2024-10-22,67,8.5
50239224,2024-07-15,24696118,9897,Amsterdam NW,2024-07-24T01:00:12.132003Z,2024-07-24T07:33:46.770637Z,2024-08-01T07:29:42.36258Z,2024-08-02T08:35:50.864005Z,2024-08-02T08:37:46.994016Z,2024-08-02T08:37:53.049151Z,2024-10-22,0,5.52
50239235,2024-07-15,20350027,8725,Almere,2024-07-24T01:00:13.856762Z,2024-07-24T07:33:45.660664Z,2024-08-01T07:29:40.196303Z,2024-08-01T08:31:14.894663Z,2024-08-01T08:33:19.048329Z,2024-08-01T08:33:32.581941Z,2024-10-22,84,9.29
50239236,2024-07-16,23642536,8727,Arnhem,2024-07-24T01:00:13.875445Z,2024-07-24T07:33:45.778637Z,2024-08-01T07:29:40.691766Z,2024-08-01T07:33:18.192031Z,2024-08-01T07:35:01.925321Z,2024-08-01T07:35:14.093337Z,2024-10-22,59,8.15
50239242,2024-07-15,24898010,8727,Arnhem,2024-07-24T01:00:14.386544Z,2024-07-24T07:33:46.006662Z,2024-08-01T07:29:46.026809Z,2024-08-01T10:14:45.639757Z,2024-08-01T11:27:35.082582Z,2024-08-01T11:27:47.652467Z,2024-10-22,100,10.0
50239277,2024-07-16,24019149,8726,Amsterdam ZO,2024-07-24T01:00:15.463385Z,2024-07-24T07:33:51.461185Z,null,2024-08-01T05:52:11.324848Z,2024-08-01T05:54:08.226273Z,2024-08-01T05:54:14.32902Z,2024-10-22,null,null
50239284,2024-07-17,24996210,8726,Amsterdam ZO,2024-07-24T01:00:15.644338Z,2024-07-24T07:33:52.446694Z,2024-08-01T07:29:44.676161Z,2024-08-01T15:09:32.691885Z,2024-08-01T15:10:27.914071Z,2024-08-01T15:10:36.104505Z,2024-10-22,71,8.7


unitId,region,year,month,avgValue,avgValueAsGrade,customersThisPeriod,promotors,detractors,passives,questionnairesCompleted,netPromotorScore
8725,Almere,2024,08,86.62,9.39,13,13,0,0,13,100.0
8726,Amsterdam ZO,2024,08,63.29,8.35,8,4,0,3,7,56.99999999999999
8727,Arnhem,2024,08,83.44,9.26,9,8,0,1,9,89.0
8753,Rotterdam/Leiden,2024,08,66.5,8.5,4,2,0,2,4,50.0
9897,Amsterdam NW,2024,08,0.0,5.52,1,0,1,0,1,-100.0
